In [1]:
from bokeh.io import show, output_notebook
from bokeh.models import Plot, TapTool, ColumnDataSource, LabelSet, StaticLayoutProvider, Circle, MultiLine
from bokeh.models.widgets import Div
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.plotting import figure
from bokeh.models.renderers import GraphRenderer, GlyphRenderer
from bokeh.layouts import layout, row, column, gridplot
from bokeh.models import CustomJS, Button, DataTable, TableColumn, Rect
from bokeh.models.glyphs import Text
from bokeh.events import ButtonClick

In [2]:
output_notebook()

Loading BokehJS ...

In [ ]:
bl_code = """
// resolve goalie playing Left

bl.disabled = true; // disable player actions
bm.disabled = true;
br.disabled = true;
b_next.disabled= false; // enable next button

// get strikers choice from last action
const foot = striker.text;

if (foot == 'left') {
    // get number of times 'left' subgame has been played
    n = parseInt(nleft.text)
    
    // if never played before
    if (n == 0) {
        const rand = Math.random(); // draw [0,1] RV
        goalprob rand = Math.random(); // draw [0,1] RV
        
        // play Left w.p 7/11 so goalie is indifferent
        if (rand < 7/11) {
            // resolve Left-play
            txt.data['text'][3] = 'Striker kicks Left; you jump Left!'
            txt.change.emit()
            // roll winning probability
            if (goalprob <= .67) {
                // resolve hit
                txt.data['text'][4] = 'Goal! (Win probability = 0.67)'
                txt.change.emit()                
            }
            // resolve miss
        }
        // else resolve Right-play
            
            txt.data['text'][3] = 'Striker kicks Right; you jump Left!'
            txt.change.emit()
    
    }
    // update number of times left subgame was played
    
} else if (foot == 'right') {
    // get number of times 'left' subgame has been played
    n = parseInt(nright.text)
    // if never played before
    if (n == 0) {
        const rand = Math.random(); // draw [0,1] RV
        // play Left w.p 7/11 so goalie is indifferent
        if (rand < 23/63) {
            // resolve Left-play
        
        }
    
    }
}

txt.data['text'][2] = 'Press Next to play another round.'
txt.change.emit()


const rand = Math.random(); // draw [0,1] RV
if (rand < 0.5) {
    // play left
    striker.text = 'left'
    const count = parseInt(nleft.text) + 1;
    nleft.text = count.toString();
    txt.data['text'][2] = 'Opponent chose a left-footed striker! Your move.'
    txt.change.emit()
} else {
    // play right
    striker.text = 'right'
    const count = parseInt(nright.text) + 1;
    nright.text = count.toString();
    txt.data['text'][2] = 'Opponent chose a right-footed striker! Your move.'
    txt.change.emit()
}
"""

In [3]:
### Define figure

    
p = figure(plot_width=400, plot_height=290, tools="", toolbar_location=None, title='FIFA 2020 Penalty Simulator',
           x_range=(0, 100), y_range=(0, 90)
          )

# hide axes and gridlines
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.outline_line_color = None

p.background_fill_color = "green"


### Goal posts and lines
p.multi_line([[17, 19, 81, 83, 17], [2, 12, 88, 98, 2], [30, 30, 70, 70]],
             [[10, 30, 30, 10, 10], [10, 65, 65, 10, 10], [10, 33, 33, 10]],
             color=["lightgreen", "lightgreen", "whitesmoke"],
             alpha=[1, 1, 1],
             line_width=4)

p.quadratic(33, 65, 67, 65, 50, 75, color='lightgreen', line_width=4)


### player sprites

# heads
src_circ = ColumnDataSource(dict(x=[50,50, 50],
                                 y=[21, 76.5, 63],
                                 size=[14, 12, 12],
                                 color=['lightblue', 'red', 'whitesmoke']))
circ = Circle(x="x", y="y", fill_color='color', line_width=2, size='size')
p.add_glyph(src_circ, circ)

# bodies
src_rect = ColumnDataSource(dict(x=[50,50],
                                 y=[15, 72],
                                 w=[4, 3],
                                 h=[6,4],
                                 color=['lightblue', 'red']))
rect = Rect(x="x", y="y", width="w", height="h", angle=0, fill_color='color', line_width=2)
p.add_glyph(src_rect, rect)

### Define buttons

b_start = Button(label="Begin", button_type="success", width_policy='min')
b_next = Button(label="Next round", button_type="success", width_policy='min', disabled=True)

bl = Button(label="Left", button_type="success", width_policy='min', disabled=True)
bm = Button(label="Middle", button_type="success", width_policy='min', disabled=True)
br = Button(label="Right", button_type="success", width_policy='min', disabled=True)

### Add text labels on graph
scr_text = ColumnDataSource({'x': [2, 70, 2, 14, 14],
                             'y': [86, 86, 5, 50, 42],
                            'text': ['Rounds played: 00',
                                     'Total score: 00',
                                     '',
                                     'Striker kicks Left; you jump Left!',
                                     'No goal! (Probability of goal was ppp)']
                            })
labels = Text(x="x", y="y", text='text', text_color="whitesmoke",
             x_offset=0, y_offset=+9,
             text_baseline='ideographic', text_align='left')

p.add_glyph(scr_text, labels)

### Store state variables in divs
rounds = Div(text='00',
             style= {'color':None, 'font-size':'1px'},
             width=10,
             height=0
            )

score = Div(text='00',
            style= {'color':None, 'font-size':'1px'},
            width=13,
            height=0
           )

# count number of times subgame has been 'left-footed'
nleft = Div(text='0',
                 style= {'color':None, 'font-size':'1px'},
                 width=0,
                 height=0
                )

# count number of times subgame has been 'right-footed'
nright = Div(text='0',
          style= {'color':None, 'font-size':'1px'},
          width=0,
          height=0
         )

# count number of times in 'left-footed' player chose 'Left'
nleft_left = Div(text='0',
                 style= {'color':None, 'font-size':'1px'},
                 width=0,
                 height=0
                )

# count number of times in 'right-footed' player chose 'Left'
nright_left = Div(text='0',
          style= {'color':None, 'font-size':'1px'},
          width=0,
          height=0
         )

# store type of current striker
striker = Div(text='none',
          style= {'color':None, 'font-size':'1px'},
          width=0,
          height=0
         )


### Define Callbacks

bb_code = """
b_start.label = 'Restart';
bl.disabled = false;
bm.disabled = false;
br.disabled = false;

// make the computer choose an action
// this first choice is random
const rand = Math.random(); // draw [0,1] RV
if (rand < 0.5) {
    // play left
    striker.text = 'left'
    const count = parseInt(nleft.text) + 1;
    nleft.text = count.toString();
    txt.data['text'][2] = 'Opponent chose a left-footed striker! Your move.'
    txt.change.emit()
} else {
    // play right
    striker.text = 'right'
    const count = parseInt(nright.text) + 1;
    nright.text = count.toString();
    txt.data['text'][2] = 'Opponent chose a right-footed striker! Your move.'
    txt.change.emit()
}
"""

b_start_click = CustomJS(args=dict(b_start=b_start,
                                   bl = bl,
                                   bm = bm,
                                   br = br,
                                   b_next = b_next,
                                   labels=labels,
                                   rounds=rounds,
                                   txt = scr_text,
                                   score=score,
                                   striker=striker,
                                   nleft=nleft,
                                   nright=nright,
                                   nleft_left=nleft_left,
                                   nright_left=nright_left
                                  ),
                        code=bb_code
                       )

b_start.js_on_click(b_start_click)

### Make layout and show
#buttons_top = row(b1, b2, b3)
buttons_bot = row(b_start, rounds, bl, bm, br, score, b_next, striker, nleft, nright, nleft_left, nright_left)
grid = gridplot([[p], [buttons_bot]], plot_width=400, plot_height=320)

show(grid)

In [108]:
scr_text.data['text'][2]

'asd'

In [99]:
nleft.text

'3'